# Evaluation & Monitoring Framework

In [4]:
#Bibliotecas
from evaluation_framework.evaluation import evaluate_classification, evaluate_regression
from evaluation_framework.cross_validation import cross_validate_model
import os
import joblib
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd
import pandas as pd
from evaluation_framework.monitoring import (
    generate_drift_report, track_experiment_csv, track_experiment_mlflow, check_alerts
)


In [5]:
# Diretórios
report_dir = 'evaluation_framework/reports'
os.makedirs(report_dir, exist_ok=True)

## Classification

In [6]:

# =========================
# Avaliação Random Forest (Classificação)
# =========================
model_path_rf = 'model_package/classification/random_forest/model.pkl'
X_test_rf = joblib.load('model_package/classification/random_forest/X_test_preprocessed.pkl')
y_test_rf = joblib.load('model_package/classification/random_forest/y_test.pkl')

metrics_rf = evaluate_classification(
    model_path=model_path_rf,
    preprocessor_path=None, 
    X_test=X_test_rf,
    y_test=y_test_rf,
    report_dir=report_dir,
    model_name='random_forest'
)

# =========================
# Avaliação XGBoost (Classificação)
# =========================
model_path_xgb = 'model_package/classification/xgboost/model.pkl'
X_test_xgb = joblib.load('model_package/classification/xgboost/X_test_preprocessed.pkl')
y_test_xgb = joblib.load('model_package/classification/xgboost/y_test.pkl')
metrics_xgb = evaluate_classification(
    model_path=model_path_xgb,
    preprocessor_path=None, 
    X_test=X_test_xgb,
    y_test=y_test_xgb,
    report_dir=report_dir,
    model_name='xgboost'
)

print("✅ Avaliação concluída!")
print("Random Forest:", metrics_rf)
print("XGBoost:", metrics_xgb)


✅ Avaliação concluída para random_forest. Métricas salvas em evaluation_framework/reports
✅ Avaliação concluída para xgboost. Métricas salvas em evaluation_framework/reports
✅ Avaliação concluída!
Random Forest: {'Accuracy': 0.6438356164383562, 'F1': 0.6754715698859775, 'ROC_AUC': 0.9102016133013437}
XGBoost: {'Accuracy': 0.0821917808219178, 'F1': 0.05581430487090864, 'ROC_AUC': 0.9201675009626493}


## Regression

In [7]:

# =========================
# Avaliação Regressão Linear (Regressão)
# =========================
model_path_linear = 'model_package/regression/linear/model.pkl'
X_test_linear = joblib.load('model_package/regression/linear/X_test_preprocessed.pkl')
y_test_linear = joblib.load('model_package/regression/linear/y_test.pkl')
metrics_linear = evaluate_regression(
    model_path=model_path_linear,
    scaler_path=None,  
    X_test=X_test_linear,
    y_test=y_test_linear,
    report_dir=report_dir,
    model_name='linear',
    log_target=True
)



# =========================
# Avaliação Ridge (Regressão)
# =========================
model_path_ridge = 'model_package/regression/ridge/model.pkl'
X_test_ridge = joblib.load('model_package/regression/ridge/X_test_preprocessed.pkl')
y_test_ridge = joblib.load('model_package/regression/ridge/y_test.pkl')


metrics_ridge = evaluate_regression(
    model_path=model_path_ridge,
    scaler_path=None,
    X_test=X_test_ridge,
    y_test=y_test_ridge,
    report_dir=report_dir,
    model_name='ridge',
    log_target=True 
)



print("✅ Avaliação concluída!")
print("Linear:", metrics_linear)
print("Ridge:", metrics_ridge)

✅ Avaliação concluída para linear. Métricas salvas em evaluation_framework/reports
✅ Avaliação concluída para ridge. Métricas salvas em evaluation_framework/reports
✅ Avaliação concluída!
Linear: {'MAE': 435.07510491449347, 'RMSE': np.float64(1604.8235973737185), 'R2': 0.18741863854601415}
Ridge: {'MAE': 436.05504949386705, 'RMSE': np.float64(1606.7495585582885), 'R2': 0.18546709789268134}


## Cross-Validation

In [8]:
# carregar dados de treino 
X_train_linear = joblib.load('model_package/regression/linear/X_train_preprocessed.pkl')
y_train_linear = joblib.load('model_package/regression/linear/y_train.pkl')
X_train_ridge = joblib.load('model_package/regression/ridge/X_train_preprocessed.pkl')
y_train_ridge = joblib.load('model_package/regression/ridge/y_train.pkl')
X_train_random_forest = joblib.load('model_package/classification/random_forest/X_train_preprocessed.pkl')
y_train_random_forest = joblib.load('model_package/classification/random_forest/y_train.pkl')
X_train_xgboost = joblib.load('model_package/classification/xgboost/X_train_preprocessed.pkl')
y_train_xgboost = joblib.load('model_package/classification/xgboost/y_train.pkl')



In [9]:
# =========================
# Cross-Validation Ridge (Regressão)
# =========================

# Instanciar modelo Ridge
model = Ridge(alpha=1.0)

cv_results = cross_validate_model(
    model=model,
    X=X_train_ridge,
    y=y_train_ridge,
    n_splits=2,
    report_dir="reports",
    model_name="ridge"
)

print(cv_results)


✅ Cross-validation concluída para ridge.
      Fold          MAE         RMSE        R2
0      1.0  1082.311227  3990.313988  0.034063
1      2.0  1183.187293  1482.925402 -1.874973
Mean   1.5  1132.749260  2736.619695 -0.920455


In [10]:
# =========================
# Cross-Validation Linear (Regressão)
# =========================

# Instanciar modelo Linear
model = LinearRegression()
cv_results = cross_validate_model(
    model=model,
    X=X_train_linear,
    y=y_train_linear,
    n_splits=5,
    report_dir="reports",
    model_name="linear"
)
print(cv_results)

✅ Cross-validation concluída para linear.
      Fold          MAE         RMSE        R2
0      1.0  1614.037926  4933.561038  0.039613
1      2.0  1478.043750  3653.257789  0.087704
2      3.0  1058.701351  1382.816342 -0.573040
3      4.0   999.874448  1330.637300 -0.364329
4      5.0   887.920202  1115.455124 -1.838893
Mean   3.0  1207.715536  2483.145519 -0.529789


In [11]:
# =========================
# Cross-Validation Random Forest (Classificação)
# =========================

model = RandomForestClassifier(n_estimators=100, random_state=42)

cv_results = cross_validate_model(
    model=model,
    X=X_test_rf,
    y=y_test_rf,
    n_splits=5,
    report_dir="reports",
    model_name="random_forest"
)
print(cv_results)

✅ Cross-validation concluída para random_forest.
      Fold  Accuracy        F1   ROC_AUC
0      1.0  0.266667  0.183333  0.671029
1      2.0  0.733333  0.742857  0.858879
2      3.0  0.733333  0.544643  0.849567
3      4.0  0.714286  0.748485  0.958333
4      5.0  0.785714  0.772436  0.942361
Mean   3.0  0.646667  0.598351  0.856034


In [12]:
# =========================
# Avaliação XGBoost (Classificação)
# =========================
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

cv_results = cross_validate_model(
    model=model,
    X=X_train_xgboost,
    y=y_train_xgboost,
    n_splits=5,
    report_dir="reports",
    model_name="xgboost"
)
print(cv_results)

c:\Users\bbelissi\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:38:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bbelissi\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:38:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bbelissi\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:38:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\bbelissi\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:38:20] WARNI

✅ Cross-validation concluída para xgboost.
      Fold  Accuracy        F1   ROC_AUC
0      1.0  0.637931  0.625600  0.903591
1      2.0  0.810345  0.763517  0.942910
2      3.0  0.758621  0.796866  0.946970
3      4.0  0.793103  0.815368  0.923749
4      5.0  0.758621  0.738122  0.933467
Mean   3.0  0.751724  0.747894  0.930137


## Drift Report

### Linear Regression

In [13]:
# Converte arrays para DataFrame com nomes genéricos
feature_names = [f"feature_{i}" for i in range(X_train_linear.shape[1])]
X_train_df = pd.DataFrame(X_train_linear, columns=feature_names)
X_test_df = pd.DataFrame(X_test_linear, columns=feature_names)

# Drift report
report_dir = "evaluation_framework/reports"
drift_results = generate_drift_report(X_train_df, X_test_df, report_dir, model_name="linear")
print(drift_results.head())

# Tracking
track_experiment_csv(report_dir, "linear", metrics_linear)
track_experiment_mlflow("linear", metrics_linear)

# Alertas
baseline = {"rmse": 0.85} 
alerts = check_alerts(baseline, metrics_linear, threshold=0.10)
print("⚠ Alertas:", alerts)


     feature     PSI  KS_stat  KS_p_value  drift_flag
0  feature_0  0.2707   0.0661      0.9434        True
1  feature_1  0.1132   0.1140      0.4006        True
2  feature_2  0.1744   0.0775      0.8423       False
3  feature_3  0.5809   0.1413      0.1744        True
4  feature_4  0.0157   0.0542      0.9908       False


c:\Users\bbelissi\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


⚠ Alertas: {}


### Ridge

In [17]:
# Converte arrays para DataFrame com nomes genéricos
feature_names = [f"feature_{i}" for i in range(X_train_ridge.shape[1])]
X_train_df = pd.DataFrame(X_train_ridge, columns=feature_names)
X_test_df = pd.DataFrame(X_test_ridge, columns=feature_names)

# Drift report
report_dir = "evaluation_framework/reports"
drift_results = generate_drift_report(X_train_df, X_test_df, report_dir, model_name="ridge")
print(drift_results.head())

# Tracking
track_experiment_csv(report_dir, "ridge", metrics_ridge)
track_experiment_mlflow("ridge", metrics_ridge)

# Alertas
baseline = {"rmse": 0.85} 
alerts = check_alerts(baseline, metrics_ridge, threshold=0.10)
print("⚠ Alertas:", alerts)

     feature     PSI  KS_stat  KS_p_value  drift_flag
0  feature_0  0.2707   0.0661      0.9434        True
1  feature_1  0.1132   0.1140      0.4006        True
2  feature_2  0.1744   0.0775      0.8423       False
3  feature_3  0.5809   0.1413      0.1744        True
4  feature_4  0.0157   0.0542      0.9908       False
⚠ Alertas: {}


### Random Forest

In [21]:
# Converte arrays para DataFrame com nomes genéricos
feature_names = [f"feature_{i}" for i in range(X_train_random_forest.shape[1])]
X_train_df = pd.DataFrame(X_train_random_forest, columns=feature_names)
X_test_df = pd.DataFrame(X_test_rf, columns=feature_names)

# Drift report
report_dir = "evaluation_framework/reports"
drift_results = generate_drift_report(X_train_df, X_test_df, report_dir, model_name="random_forest")
print(drift_results.head())

# Tracking
track_experiment_csv(report_dir, "random_forest", metrics_rf)
track_experiment_mlflow("random_forest", metrics_rf)

# Alertas
baseline = {"F1": 0.75} 
alerts = check_alerts(baseline, metrics_rf, threshold=0.10)
print("⚠ Alertas:", alerts)

     feature     PSI  KS_stat  KS_p_value  drift_flag
0  feature_0  0.1640   0.0814      0.7964       False
1  feature_1  0.2649   0.1273      0.2737        True
2  feature_2  0.4148   0.0968      0.6066        True
3  feature_3  0.1218   0.1213      0.3272        True
4  feature_4  0.1354   0.1124      0.4143        True
⚠ Alertas: {'F1': False}


### XGBoost

In [22]:
# Converte arrays para DataFrame com nomes genéricos
feature_names = [f"feature_{i}" for i in range(X_train_xgboost.shape[1])]
X_train_df = pd.DataFrame(X_train_xgboost, columns=feature_names)
X_test_df = pd.DataFrame(X_test_xgb, columns=feature_names)

# Drift report
report_dir = "evaluation_framework/reports"
drift_results = generate_drift_report(X_train_df, X_test_df, report_dir, model_name="xgboost")
print(drift_results.head())

# Tracking
track_experiment_csv(report_dir, "xgboost", metrics_xgb)
track_experiment_mlflow("xgboost", metrics_xgb)

# Alertas
baseline = {"F1": 0.75} 
alerts = check_alerts(baseline, metrics_xgb, threshold=0.10)
print("⚠ Alertas:", alerts)

     feature     PSI  KS_stat  KS_p_value  drift_flag
0  feature_0  0.1640   0.0814      0.7964       False
1  feature_1  0.2649   0.1273      0.2737        True
2  feature_2  0.4148   0.0968      0.6066        True
3  feature_3  0.1218   0.1213      0.3272        True
4  feature_4  0.1354   0.1124      0.4143        True
⚠ Alertas: {'F1': True}
